## Login to Low-side domain as a data scientist

In [1]:
import syft as sy
import pandas as pd

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/carmenpopa/.hagrid/quickstart


In [2]:
guest_domain_client = sy.login(url='http://localhost:8081', email='ana.banana@uni.org', password='student')
guest_domain_client

Logged into Modest Karp as <ana.banana@uni.org>


<SyftClient - Modest Karp <d87e4efcc9e8494b826df57b8670a2b3>: HTTPConnection: http://localhost:8081>

## Check the datasets in the domain

In [3]:
guest_domain_client.datasets

,type,id,name,url
0,syft.service.dataset.dataset.Dataset,3d404c84206b40b4a93f8d1c8ad0dd3a,Bubble tea mock data,None


In [4]:
dataset = guest_domain_client.datasets[0]
dataset

```python
Syft Dataset: Bubble tea mock data
Assets:
	mock_bubble_tea_data: Mock data for bubble tea consumption
Citation: Carmen Popa
Description: The fake data to show bubble tea trends for consumers

```

We can access the dataset assets (a.k.a the actual data) by running the following command.

In [5]:
dataset.assets

,key,type,id
0,mock_bubble_tea_data,syft.service.dataset.dataset.Asset,dfbfa3dc97c241d6ba3e4496dc2835fb


In [6]:
mock = dataset.assets[0].mock
print(mock.shape)

mock_df = mock.syft_action_data
mock_df.head()

(4, 4)


,drink_name,sugar_level,has_pearls,customer_ratings
0,brown_sugar,0.7,True,4.9
1,thai_milk,0.3,True,4.6
2,mango_coconut,0.9,False,3.5
3,strawberry_cheese,0.5,True,4.3


In [7]:
mock_df[0:2]

,drink_name,sugar_level,has_pearls,customer_ratings
0,brown_sugar,0.7,True,4.9
1,thai_milk,0.3,True,4.6


In [8]:
mock_df["sugar_level"].max()

0.9

## Writing queries

 **Which bubble tea flavour has the best customer ratings?**

At first, let's check the true result in the mock data so that we can validate our custom function later on. 

In [9]:
mock_df.iloc[mock_df['customer_ratings'].argmax()]["drink_name"]

'brown_sugar'

Now that we know the true result, we will create our custom function which, if approved, will be used on real data on the high-side domain.

In [10]:
@sy.syft_function(input_policy=sy.ExactMatch(drink_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def most_liked_bubble_tea(drink_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    # your code:
    df = drink_data
    most_liked_drink_row = df.iloc[mock_df['customer_ratings'].argmax()]
    return (# str(most_liked_drink_row["drink_name"]),
            float(most_liked_drink_row["customer_ratings"]),
            float(noise)
           )

Let's verify that the function and policy works on the mock data and returns the expected result.

In [11]:
result = most_liked_bubble_tea(drink_data=mock)
result

(4.9, -3.534334644938589)

The function works as expected.

## Inspecting the attributes of the code & policy

In [12]:
most_liked_bubble_tea.kwargs

{NodeView(node_name='Modest Karp', verify_key=04137bfea77b4c86492df2b9849190bd4a19b7a2833b3c500dd37fef6957615c): {'drink_data': <UID: 4d6b7de22de8464ea7d53ec6a1ee5788>}}

In [13]:
most_liked_bubble_tea.input_policy_type

syft.service.policy.policy.ExactMatch

In [14]:
most_liked_bubble_tea.output_policy_type

syft.service.policy.policy.OutputPolicyExecuteOnce

In [15]:
most_liked_bubble_tea.code

'@sy.syft_function(input_policy=sy.ExactMatch(drink_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef most_liked_bubble_tea(drink_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    # your code:\n    df = drink_data\n    most_liked_drink_row = df.iloc[mock_df[\'customer_ratings\'].argmax()]\n    return (# str(most_liked_drink_row["drink_name"]),\n            float(most_liked_drink_row["customer_ratings"]),\n            float(noise)\n           )\n'

## Submit code for review on the Data Owner side

In [16]:
new_project = sy.Project(name="My first query on bubble tea data")
proj_desc = """Hi, I want to know which bubble tea flavour is the most liked"""

In [17]:
guest_domain_client.api.services.code.request_code_execution(most_liked_bubble_tea)

```python
class Request:
  id: str = 00bcdc0dfa8e49c2bd6872b14260ba5a
  requesting_user_verify_key: str = 4ea8053fe7417899e19a84448a46ffceed451d06ed90eef0bc5b1f2aa9e10143
  approving_user_verify_key: str = None
  request_time: str = 2023-05-03 20:50:29
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = d87e4efcc9e8494b826df57b8670a2b3
  request_hash: str = "ee26272cbe1af80905ca61d73b00010d1a805efc7f803731701ce8da2774d6fd"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [18]:
guest_domain_client.api.services.code.request_code_execution(most_liked_bubble_tea)
guest_domain_client.code

,type,id,status,service_func_name
0,syft.service.code.user_code.UserCode,3b72f052e2a440179cecab37e723af6b,"{NodeView(node_name='Modest Karp', verify_key=...",most_liked_bubble_tea


In [19]:
new_project.set_description(proj_desc)
submitted_code = guest_domain_client.code[0]
submitted_code

```python
class UserCode:
  id: str = 3b72f052e2a440179cecab37e723af6b
  node_uid: str = None
  user_verify_key: str = 4ea8053fe7417899e19a84448a46ffceed451d06ed90eef0bc5b1f2aa9e10143
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(drink_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def most_liked_bubble_tea(drink_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    # your code:
    df = drink_data
    most_liked_drink_row = df.iloc[mock_df['customer_ratings'].argmax()]
    return (# str(most_liked_drink_row["drink_name"]),
            float(most_liked_drink_row["customer_ratings"]),
            float(noise)
           )
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='Modest Karp', verify_key=04137bfea77b4c86492df2b9849190bd4a19b7a2833b3c500dd37fef6957615c): {'drink_data': <UID: 4d6b7de22de8464ea7d53ec6a1ee5788>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.service.policy.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_most_liked_bubble_tea_4ea8053fe7417899e19a84448a46ffceed451d06ed90eef0bc5b1f2aa9e10143_b4bd04f2a5bd30a85536f05f53741ceb9121c015867cf1fc684d3217df0b60d4(drink_data):

    def most_liked_bubble_tea(drink_data):
        import pandas as pd
        from opendp.mod import enable_features
        enable_features('contrib')
        from opendp.measurements import make_base_laplace
        aggregate = 0.0
        base_lap = make_base_laplace(scale=5.0)
        noise = base_lap(aggregate)
        df = drink_data
        most_liked_drink_row = df.iloc[mock_df['customer_ratings'].argmax()]
        return (float(most_liked_drink_row['customer_ratings']), float(noise))
    result = most_liked_bubble_tea(drink_data=drink_data)
    return result"
  service_func_name: str = "most_liked_bubble_tea"
  unique_func_name: str = "user_func_most_liked_bubble_tea_4ea8053fe7417899e19a84448a46ffceed451d06ed90eef0bc5b1f2aa9e10143_b4bd04f2a5bd30a85536f05f53741ceb9121c015867cf1fc684d3217df0b60d4"
  user_unique_func_name: str = "user_func_most_liked_bubble_tea_4ea8053fe7417899e19a84448a46ffceed451d06ed90eef0bc5b1f2aa9e10143"
  code_hash: str = "b4bd04f2a5bd30a85536f05f53741ceb9121c015867cf1fc684d3217df0b60d4"
  signature: str = (drink_data)
  status: str = {NodeView(node_name='Modest Karp', verify_key=04137bfea77b4c86492df2b9849190bd4a19b7a2833b3c500dd37fef6957615c): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['drink_data']
  enclave_metadata: str = None

```

In [20]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)
new_project.changes

,type,id
0,syft.service.project.project.ObjectPermissionC...,532e01e8b5814360bb1803d84298db45


In [21]:
new_project

```python
class ProjectSubmit:
  id: str = None
  name: str = "My first query on bubble tea data"
  description: str = "Hi, I want to know which bubble tea flavour is the most liked"
  changes: str = [syft.service.project.project.ObjectPermissionChange]

```

Now we are ready to submit the code.

In [22]:
guest_domain_client.submit_project(new_project)

<class 'syft.service.response.SyftSuccess'>: Project Submitted

**At this point, we will go back to the data-owner-workflow notebook to review the submitted code and either approve or reject it.**